# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [25]:
# Initial imports

import os
import pandas as pd
from newsapi import NewsApiClient
from dotenv import load_dotenv
load_dotenv("example.env")

import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from nltk.corpus import stopwords, reuters
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt

# Code to download corpora
import nltk
nltk.download('wordnet')
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')


lemmatizer = WordNetLemmatizer()


%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jc/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to /Users/jc/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# Read your api key environment variable
api_key = os.getenv("news_api")

In [10]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [11]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_top_headlines(q="bitcoin", language="en", country="us")
bitcoin_headlines

{'status': 'ok',
 'totalResults': 1,
 'articles': [{'source': {'id': 'cbs-news', 'name': 'CBS News'},
   'author': None,
   'title': 'Why is the price of Bitcoin and other cryptocurrencies falling? - CBS News',
   'description': 'Digital currencies slumped after Chinese banking authorities moved to clamp down on their usage.',
   'url': 'https://www.cbsnews.com/news/crypto-price-bitcoin-down-2021-05-22/',
   'urlToImage': 'https://cbsnews3.cbsistatic.com/hub/i/r/2021/05/19/8edc83f1-a9ea-4f02-b48e-0adb30b98046/thumbnail/1200x630/19382be932428563d70d738adcbe7e9a/2021-05-19t203900z-541777464-rc28jn928g9j-rtrmadp-3-crypto-currency-bitcoin.jpg',
   'publishedAt': '2021-05-22T10:59:29Z',
   'content': 'The price of Bitcoin fell as much as 29% Wednesday after the China Banking Association warned member banks of risks associated with digital currencies. Other digital currencies suffered sharp decline… [+7661 chars]'}]}

In [12]:
# Fetch the Ethereum news articles
ethereum_headlines = newsapi.get_top_headlines(q="ethereum", language="en", country="us")
ethereum_headlines

{'status': 'ok', 'totalResults': 0, 'articles': []}

In [13]:
# Fetch all the news about Bitcoin
bitcoin_headlines = newsapi.get_everything(
    q="bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Bitcoin: {bitcoin_headlines['totalResults']}")

# Show sample article
bitcoin_headlines["articles"][0]

Total articles about Bitcoin: 11914


{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'https://www.engadget.com/about/editors/richard-lawler',
 'title': "Tesla 'suspends' Bitcoin car purchases citing environmental impact",
 'description': "You can't buy a Tesla with Bitcoin anymore..",
 'url': 'https://www.engadget.com/elon-musk-bitcoin-221708146.html',
 'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2021-05/a0f90c30-b36f-11eb-aff6-04fb28cf2f4b',
 'publishedAt': '2021-05-12T22:17:08Z',
 'content': 'Just weeks after Tesla started accepting Bitcoin as currency for cars, Elon Musk revealed in a tweet that it will "suspend" the effort. According to the release (Tesla does not appear to have a funct… [+768 chars]'}

In [14]:
#VADER Sentiment Score
article = bitcoin_headlines["articles"][0]

In [15]:
## each iteration

text = article["content"]
date = article["publishedAt"][:10]
sentiment = analyzer.polarity_scores(text)
compound = sentiment["compound"]
pos = sentiment["pos"]
neu = sentiment["neu"]
neg = sentiment["neg"]

print(f'positive score: {pos}')
print(f'neutral score: {neu}')
print(f'negative score: {neg}')

positive score: 0.071
neutral score: 0.929
negative score: 0.0


In [16]:
# Describe the Bitcoin Sentiment
bitcoin_sentiments = []

for article in bitcoin_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
bitcoin_df = pd.DataFrame(bitcoin_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
bitcoin_df = bitcoin_df[cols]
bitcoin_df.head()


,date,text,compound,positive,negative,neutral
0,2021-05-12,Just weeks after Tesla started accepting Bitco...,0.3818,0.071,0.00,0.929
1,2021-05-07,Shares of Square are up more than 6% today aft...,0.2960,0.058,0.00,0.942
2,2021-04-27,Tesla’s relationship with bitcoin is not a dal...,0.0000,0.000,0.00,1.000
3,2021-05-12,Image: Tesla\r\n\n \n\n Tesla has stopped acce...,0.4939,0.134,0.05,0.816
4,2021-04-23,Cryptocurrency prices continued to tumble Frid...,0.0000,0.000,0.00,1.000


In [17]:
# Fetch all the news about Ethereum
ethereum_headlines = newsapi.get_everything(
    q="ethereum",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Ethereum: {ethereum_headlines['totalResults']}")

# Show sample article
ethereum_headlines["articles"][0]


Total articles about Ethereum: 4675


{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
 'author': 'Manish Singh',
 'title': 'Vitalik Buterin donates $1 billion worth of ‘meme coins’ to India Covid Relief Fund',
 'description': 'Vitalik Buterin, the creator of Ethereum, on Wednesday donated Ethereum and “meme coins” worth $1.5 billion in one of the largest-ever individual philanthropy efforts. Buterin transferred 500 ETH and over 50 trillion SHIB (Shiba Inu), a meme coin, worth aroun…',
 'url': 'http://techcrunch.com/2021/05/12/vitalik-buterin-donates-1-billion-worth-of-meme-coins-to-india-covid-relief-fund/',
 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2017/09/vitalik-buterin-147a2566.jpg?w=600',
 'publishedAt': '2021-05-12T22:46:10Z',
 'content': 'Vitalik Buterin, the creator of Ethereum, on Wednesday donated Ethereum and meme coins worth $1.5 billion in one of the largest-ever individual philanthropy efforts.\r\nButerin transferred 500 ETH and … [+1667 chars]'}

In [18]:
#VADER Sentiment Score
article = ethereum_headlines["articles"][0]

In [19]:
## each iteration

text = article["content"]
date = article["publishedAt"][:10]
sentiment = analyzer.polarity_scores(text)
compound = sentiment["compound"]
pos = sentiment["pos"]
neu = sentiment["neu"]
neg = sentiment["neg"]

print(f'positive score: {pos}')
print(f'neutral score: {neu}')
print(f'negative score: {neg}')

positive score: 0.06
neutral score: 0.94
negative score: 0.0


In [20]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiments = []

for article in ethereum_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ethereum_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
ethereum_df = pd.DataFrame(ethereum_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
ethereum_df = ethereum_df[cols]
ethereum_df.head()

,date,text,compound,positive,negative,neutral
0,2021-05-12,"Vitalik Buterin, the creator of Ethereum, on W...",0.2263,0.060,0.000,0.940
1,2021-05-04,Their investors call them disruptive innovator...,-0.2144,0.054,0.111,0.835
2,2021-05-15,Solana isn’t known yet outside of the crypto c...,0.5499,0.106,0.000,0.894
3,2021-05-01,New York (CNN Business)Bitcoin prices continue...,0.0000,0.000,0.000,1.000
4,2021-05-03,"The creators behind CryptoPunks, one of the mo...",0.4754,0.091,0.000,0.909


In [21]:
bitcoin_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.00000
mean,0.066883,0.055280,0.039110,0.90564
std,0.397988,0.067863,0.053804,0.08300
min,-0.762700,0.000000,0.000000,0.67700
25%,-0.058450,0.000000,0.000000,0.85450
50%,0.000000,0.019500,0.000000,0.91250
75%,0.381800,0.081750,0.070250,1.00000
max,0.845500,0.275000,0.219000,1.00000


In [22]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.054273,0.047440,0.035430,0.907120
std,0.336819,0.055704,0.059887,0.122004
min,-0.918600,0.000000,0.000000,0.000000
25%,-0.019300,0.000000,0.000000,0.857250
50%,0.000000,0.046000,0.000000,0.928000
75%,0.238025,0.077250,0.067250,1.000000
max,0.822500,0.256000,0.289000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin has the highest mean positive score: 0.06442

Q: Which coin had the highest compound score?

A: Bitcoin had the highest compound score

Q. Which coin had the highest positive score?

A: Bitcoin had the highest positive score

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [23]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/jc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
#words = word_tokenize(article)
sw = set(stopwords.words('english'))
print(sw)

{"haven't", 'just', 'down', 'out', 'that', 'or', 'not', "wasn't", 'she', 'very', 'wasn', 'until', 'yourself', 'their', 'about', 'too', 's', 'some', 'his', "shan't", 'now', 'itself', 'd', "hadn't", 'for', 'of', 'in', "should've", 'ourselves', "won't", 'no', 'wouldn', 'you', "she's", 'above', 'me', 'herself', 'o', 'him', 'don', 'is', 'doesn', 'all', 'needn', 'before', 'shan', 'had', 'whom', 'off', "isn't", 'an', 'other', 'during', 'have', 'y', 'again', 'but', 've', 'being', 'hasn', 'yourselves', 'few', 'the', 'those', "mustn't", 'we', 'myself', "don't", 'own', "you'll", 'hadn', 'under', 'further', 'couldn', 'having', 'weren', "didn't", 'when', 'ain', "it's", 'won', 'did', 'been', 'these', 'while', 'your', 'shouldn', 'as', 'was', 'do', 'against', "shouldn't", 'after', 'mightn', 'theirs', 'they', 'does', 'ma', 'at', 'then', 'were', 'doing', 'both', 'am', "aren't", 'which', 'm', 'through', 're', 'be', 'hers', 'there', 'than', 'i', 'll', 'can', 'who', "wouldn't", 'once', 'a', 'between', 'onl

In [44]:
# Expand the default stopwords list if necessary
#sw_addon = {'said', 'mln', 'kilolitres','kl'}
#second_result = [word.lower() for word in words if word.lower() not in sw.union(sw_addon)]
# NLTK list of stopwords
#print(second_result)

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
   
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    output = [word.lower() for word in lem if word.lower() not in sw]
    
    # Remove the stop words
    
    
    return tokens

In [13]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [45]:
from collections import Counter
from nltk import ngrams

In [47]:
def process_text(doc):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', article)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return output

In [51]:
article = ethereum_headlines["articles"][0]

In [52]:
# Generate the Bitcoin N-grams where N=2
processed = process_text(article)

bigram_counts = Counter(ngrams(processed, n=2))
print(dict(bigram_counts))

TypeError: expected string or bytes-like object

In [17]:
# Generate the Ethereum N-grams where N=2
bigram_counts = Counter(ngrams(processed, n=2))
print(dict(bigram_counts))

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [31]:
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [28]:
# Generate the Bitcoin word cloud
ids = reuters.fileids(categories='gold')
corpus = [reuters.raw(i) for i in ids]

def process_text(doc):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', doc)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return ' '.join(output)

In [ ]:
# Process text for wordcloud creation
big_string = ' '.join(corpus)
input_text = process_text(big_string)

In [ ]:
wc = WordCloud().generate(input_text)
plt.imshow(wc)

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---